<a href="https://colab.research.google.com/github/Kevoen/Google_Colab_Rep/blob/master/Datawhale_%E9%9B%B6%E5%9F%BA%E7%A1%80%E5%85%A5%E9%97%A8CV%E8%B5%9B%E4%BA%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive

'0720掌控习惯 .gdoc'		  images	    网络
 Colab_Notebooks		  obj.data	    网络转存文件夹
 Colad_Notebooks		  ProjectFiles	    资源
 Deep-Learning-with-PyTorch.pdf   Share-2020Books


In [ ]:
# 显示
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# 上传文件
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# 下载文件 
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
upload()

Saving mchar_data_list_0515.csv to mchar_data_list_0515.csv
saved file mchar_data_list_0515.csv


In [ ]:
#读取数据
import pandas as pd
data_list = pd.read_csv("/content/mchar_data_list_0515.csv")
data_list.head(10)

In [ ]:
data_list['link']

0    http://tianchi-competition.oss-cn-hangzhou.ali...
1    http://tianchi-competition.oss-cn-hangzhou.ali...
2    http://tianchi-competition.oss-cn-hangzhou.ali...
3    http://tianchi-competition.oss-cn-hangzhou.ali...
4    http://tianchi-competition.oss-cn-hangzhou.ali...
5    http://tianchi-competition.oss-cn-hangzhou.ali...
Name: link, dtype: object

In [18]:
%cd /content/
%cd ../mydrive/Colab_Notebooks/TianChi/

/content
/content/drive/My Drive/Colab_Notebooks/TianChi


In [ ]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_train.zip
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_train.json
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_val.zip
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_val.json
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_test_a.zip
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531795/mchar_sample_submit_A.csv

In [17]:
%ls

'Datawhale 零基础入门CV赛事.ipynb'   mchar_test_a.zip   mchar_val/
 __MACOSX/                           mchar_train/       mchar_val.json
 mchar_sample_submit_A.csv           mchar_train.json   mchar_val.zip
 mchar_test_a/                       mchar_train.zip


In [ ]:
#!unzip -d ./mchar/train/ mchar_train.zip
#!unzip mchar_train.zip
#!unzip mchar_val.zip
!unzip mchar_test_a.zip

In [19]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

%pylab inline

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

Populating the interactive namespace from numpy and matplotlib


## 定义数据集

In [20]:
class SVHNDataset(Dataset):

  def __init__(self, img_path, img_label, transform=None):  #初始化函数 内部参数初始化
    self.img_path = img_path
    self.img_label = img_label
    if transform is not None:
      self.transform = transform
    else:
      self.transform = None
  
  def __getitem__(self, index):          #获取Dataset数据
    img = cv2.imread(self.img_path[index])    #使用opencv读取图像
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #cv2.cvtColor()函数返回的是一个img数组，后面需要Image类；
    #img = Image.open(self.img_path[index]).convert('RGB')
    if self.transform is not None:
      img = self.transform(img)

    lbl = self.img_label[index]   
    lbl = lbl + (5 - len(lbl)) * [10]   #定长字符填充

    # 原始SVHN中类别10为数字0
    #lbl = np.array(self.img_label[index], dtype=np.int)
    #lbl = list(lbl)  + (5 - len(lbl)) * [10]

    return img, torch.from_numpy(np.array(lbl[:5]))  #使用torch.from_numpy()将numpy转换成张量

  def __len__(self):
    return len(self.img_path)

## 定义训练数据和验证数据的Dataset

In [36]:
train_path = glob.glob('./mchar_train/*.png') #加载train data路径
train_path.sort()
train_json = json.load(open('./mchar_train.json'))#加载标签json
train_label = [train_json[x]['label'] for x in train_json]#加载便签
print("train_path:",len(train_path),"train_label:",len(train_label))

train_loader = torch.utils.data.DataLoader(      #设置Pytorch的Dataset数据加载器
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                  transforms.Resize((64, 120)),   #缩放到固定大小
                  transforms.RandomCrop((60, 120)),   #切割中心点的位置随机选取
                  transforms.ColorJitter(0.3, 0.3, 0.2), #随机颜色变换
                  transforms.RandomRotation(10),    #随机旋转
                  transforms.ToTensor(),     # 将图片转换为pytorch的tesntor
                  transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) # 对图像像素进行归一化                   
                ])),
    batch_size = 50,
    shuffle = True,
    num_workers = 10
)

val_path = glob.glob('./mchar_val/*.png')
val_path.sort()
val_json = json.load(open('./mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print("val_path:",len(val_path),"val_label:",len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                  transforms.Resize((64, 120)),
                  # transforms.ColorJitter(0.3, 0.3, 0.2),
                  # transforms.RandomRotation(10),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])                    
                ])),
    batch_size = 50,
    shuffle = False,
    num_workers = 10                               
)

# for data in train_loder:
#   break

#[img, label] = data
# print(img.size(), label.size())
#print(label)

train_path: 30000 train_label: 30000
val_path: 10000 val_label: 10000


构建模型

In [31]:
class SVHN_Model1(nn.Module):
  def __init__(self):
    super(SVHN_Model1, self).__init__()
    #CNN提取特征模块

    model_conv = models.resnet18(pretrained=True)
    model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
    model_conv = nn.Sequential(*list(model_conv.children())[:-1])
    self.cnn = model_conv

    self.fc1 = nn.Linear(512, 11)
    self.fc2 = nn.Linear(512, 11)
    self.fc3 = nn.Linear(512, 11)
    self.fc4 = nn.Linear(512, 11)
    self.fc5 = nn.Linear(512, 11)

  def forward(self, img):
    feat = self.cnn(img)
    # print(feat.shape)
    feat = feat.view(feat.shape[0], -1)
    c1 = self.fc1(feat)
    c2 = self.fc2(feat)
    c3 = self.fc3(feat)
    c4 = self.fc4(feat)
    c5 = self.fc5(feat)
    return c1,c2,c3,c4,c5

## 每个Epoch的训练、验证、测试代码

In [32]:
def train(train_loader, model, criterion, optimizer, epoch):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    
    for i, (input, target) in enumerate(train_loader):
        if use_cuda:
            input = input.cuda()
            target = target.cuda()

        c0, c1, c2, c3, c4 = model(input)
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])
        
        # loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                if use_cuda:
                    output = np.concatenate([
                        c0.data.cpu().numpy(), 
                        c1.data.cpu().numpy(),
                        c2.data.cpu().numpy(), 
                        c3.data.cpu().numpy(),
                        c4.data.cpu().numpy()], axis=1)
                else:
                    output = np.concatenate([
                        c0.data.numpy(), 
                        c1.data.numpy(),
                        c2.data.numpy(), 
                        c3.data.numpy(),
                        c4.data.numpy()], axis=1)
                
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

In [38]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
best_loss = 1000.0
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr)  #优化器，Adam自适应调整学习率
# 是否使用GPU
use_cuda = True
if use_cuda:
    model = model.cuda()

for epoch in range(20):
    
    train_loss = train(train_loader, model, criterion, optimizer, epoch)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    
    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2} \t Val Acc: {3}'.format(epoch, train_loss, val_loss, val_char_acc))
    # print('Val Acc', val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        # print('Find better model in Epoch {0}, saving model.'.format(epoch))
        torch.save(model.state_dict(), './model.pt')

Epoch: 0, Train loss: 3.570577224691709 	 Val loss: 3.467509013414383 	 Val Acc: 0.3503
Epoch: 1, Train loss: 2.236784227887789 	 Val loss: 3.0147114324569704 	 Val Acc: 0.4339
Epoch: 2, Train loss: 1.8742641349633535 	 Val loss: 2.8130609297752383 	 Val Acc: 0.4647
Epoch: 3, Train loss: 1.6511709041396776 	 Val loss: 2.8379648435115814 	 Val Acc: 0.462
Epoch: 4, Train loss: 1.482074794570605 	 Val loss: 2.7579031616449354 	 Val Acc: 0.49
Epoch: 5, Train loss: 1.3397448036074637 	 Val loss: 2.5652968788146975 	 Val Acc: 0.5117
Epoch: 6, Train loss: 1.2293370397885641 	 Val loss: 2.603809491991997 	 Val Acc: 0.5074
Epoch: 7, Train loss: 1.1353463484346866 	 Val loss: 2.780683387517929 	 Val Acc: 0.505
Epoch: 8, Train loss: 1.05139728029569 	 Val loss: 2.6150356978178024 	 Val Acc: 0.5407
Epoch: 9, Train loss: 0.9564973652859529 	 Val loss: 2.760325657129288 	 Val Acc: 0.5288
Epoch: 10, Train loss: 0.889187194382151 	 Val loss: 2.8139515709877014 	 Val Acc: 0.5343
Epoch: 11, Train loss: 

## 预测并生成提交文件

In [ ]:
test_path = glob.glob('./mchar_test/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
          transforms.Compose([
            transforms.Resize((60, 120)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=40,
    shuffle=False,
    num_workers=10,
)

In [ ]:
# 加载保存的最优模型生成提交文件
model.load_state_dict(torch.load('model.pt'))

test_predict_label = predict(test_loader, model, 1)
print(test_predict_label.shape)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))

import pandas as pd
df_submit = pd.read_csv('../../../dataset/tianchi_SVHN/test_A_sample_submit.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('submit.csv', index=None)  